In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
colors = ['#B90276','#50237F', '#005691', '#008ECF','#E20015', '#00A8B0', '#78BE20', '#006249', '#525F6B']

sns.set_palette(sns.color_palette(colors))


# REading the Data

In [1]:
data = pd.read_csv('/kaggle/input/telecom-users-dataset/telecom_users.csv')
df = pd.DataFrame(data)
df.head()

# Dropping unwanted columns

In [1]:
df = df.drop(['Unnamed: 0','customerID'], 1)

In [1]:
df.info()

In [1]:
def convert_x(x):
    try:
        return float(x)
    except:
        return np.NaN
df['TotalCharges'] = df['TotalCharges'].apply(convert_x)

In [1]:
object1 = df.select_dtypes(include = 'object').columns

# What are the unique values in our Categorical columns?

In [1]:
for col in object1:
    print('\n', col, '\n', df[col].unique(),'\n')

# Are there any null values?

In [1]:
df.isnull().sum()

# Filling the Null Values with median

In [1]:
df.TotalCharges = df['TotalCharges'].fillna(df['TotalCharges'].median())

In [1]:
df.TotalCharges.shape

# Count Plot of our Target Column "Churn shown that the data is imblanced
***In Such cases we have to under sample or over sample the data.***

***Since the data has more count of people NOT churning out, this will train the model to identify people who are NOT churning***
***ML Model work well when there is Balance data i.e. churn 'yes' & 'no' are almost equal***

In [1]:
sns.countplot(x='Churn', data = df)

# As seen above traget data is imbalanced

# Visualization of basis of Churn

In [1]:
i = 1
plt.figure(figsize = (15,25))
for col in object1:
    plt.subplot(6,3,i)
    sns.countplot(x = 'Churn',hue = df[col] , data = df)
    i +=1

In [1]:
float1 = df.select_dtypes(exclude = 'object').columns
float1

In [1]:
i = 1
plt.figure(figsize = (15,10))
for col in float1:
    plt.subplot(2,2,i)
    sns.histplot(x = df[col])
    i +=1

# FacetGrid on Basis of MonthlyCharges & other parameters

In [1]:
plt.figure(figsize=(10,7))
for col in object1:
    g = sns.FacetGrid(data, col=col, hue="Churn", palette = 'dark')
    g.map(sns.kdeplot, "MonthlyCharges",alpha=.7)
    plt.figure(figsize=(15,7))
    g.add_legend()

# FacetGrid on Basis of Tenure & other parameters

In [1]:
plt.figure(figsize=(10,7))
for col in object1:
    g = sns.FacetGrid(data, col=col, hue="Churn")
    g.map(sns.kdeplot, "tenure",alpha=.7)
    plt.figure(figsize=(15,7))
    g.add_legend()

***Remaping Churns in to 1 & 0. 1 is Yes, 0 is no***

***I am doing this because in the next step i will be useing get dummies function, if I don't do it in get dummies I will get two columns i.e. Churn_yes, & Churn_no. Then it will be difficult to set the target in the ML alogorithms***

In [1]:
df.Churn = df['Churn'].map({'Yes':1, 'No':0})

In [1]:
df1 = pd.get_dummies(df)

In [1]:
from sklearn. preprocessing import LabelEncoder, StandardScaler
#le = LabelEncoder()
sc = StandardScaler()

    
df1.head()

# Corelation plot after using get dummies

In [1]:
plt.figure(figsize = (15,10))
corr = df1.corr()
sns.heatmap(corr, linewidth = 0.7, cmap = 'coolwarm')

In [1]:
X = df1.drop(['Churn'],1)
y = df1.Churn

# Under Sampling the Data

In [1]:
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(random_state=42, replacement=True)# fit predictor and target variable
X_rus, y_rus = rus.fit_resample(X, y)

In [1]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 43)

In [1]:
scaler = StandardScaler()
scaler.fit(X_train, y_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Fitting the data on different classification Model

In [1]:
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score , plot_roc_curve
from xgboost import XGBClassifier

from sklearn import metrics
from sklearn.metrics import mean_squared_error
rf = RandomForestClassifier(random_state=43)
ad = AdaBoostClassifier(base_estimator =rf)
dt = DecisionTreeClassifier()
kn = KNeighborsClassifier()
rbf = RBF()
gp = GaussianProcessClassifier(1.0 * RBF(1.0))
mlp = MLPClassifier(alpha=1, max_iter=1000)
gnb = GaussianNB()
svc = SVC(random_state = 43, C = 10, gamma = 0.1, kernel ='rbf')
xgb = XGBClassifier(label_encoder = False)

models = [rf,ad, dt, kn, svc, mlp]
for model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    scores = cross_val_score(model, X, y, cv=5).mean().round(3)
    accuracy = metrics.classification_report(y_test, y_pred)
    #f1score = metrics.f1_score(y_test, y_pred).round(3)
    print(model, '\n', 'REPORT:','\n', accuracy,'\n', 'mean_score:',scores, '\n' )
